# Fine Tuning

We are going to try and fine tune a FCN classifier so to segment cells from histopathology scans. Our implementation is based on the implementation based on FCN.  


The fine tuning wil be addressed in four steps. Preparing the data in the form of blobs, so as to be caffee compatible but also in three model training. We will train with a 32 stride, take the parameters and plug them into a finer 16 stride model, and finally the same protocole is used to go a 8 stride finner model.

We will use the pre-train models available in the caffee zoo. We will remove each final layer and fine tune it via back propagation. 




# Preparing the data

Under the library caffee, data is provided to model under the form of blobs. A blob is a 4 dimensinal array where each channels represents: (num, channel, height, width). We will be providing cell segmentation data from histopathology slides. 
It can accept several types: HDF5, LMDB, etc..

In [1]:
import lmdb
import caffe

# Preparing the 32 stride model: Pascal voc example

The architecture and pre-trained weight parameters can be found on the FCN github page. The architecture is based on the famous AlexNet implementation, the authors performed "net surgery" to transform the fully connected layers (final layers of the AlexNet model) into fully convolutionnal layers. These layers have the particularity of accepting any given size of inputs. Once these layers "convoloutionnized" , they added an upsampling and a deconvolution layer as to improve the accuracy of the model, indeed, AlexNet models provided the "what" but does not initially provide the where. Bridges were also added in order to combine low lever features to higher level features.